<a href="https://colab.research.google.com/github/vsolodkyi/NeuralNetworks_SkillBox/blob/main/module_13/mid_ml_nlp_les_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Урок 4. Векторизация слов: Word2Vec

Это более новый алгоритм, чем BoW.

Алгоритм предполагает, что мы  уже разбили документы на токены и готовы скормить наши токены нейросети, которая сделает из каждого токена плотный вектор-эмбеддинг (см. ниже). И в этом уроке мы разберемся с тем, как у нейросети это получится.

При подходе BoW мы представляем каждый документ в виде разреженного вектора-строки, где размерность вектора соответствует количеству токенов в словаре. 

Нетрудно заметить, что при таком подходе игнорируется контекст, в котором находится слово. Например, в двух предложениях "король издал указ" и  "правитель издал указ" слова *король* и *правитель* являются синонимами, потому что используются в одинаковом контекста. Подход *BoW* не сможет уловить отношение синонимии.



Эту проблему решает подход `Word Embedding`, при котором каждое слово представляет собой вектор большой размерности (обычно несколько сотен). В отличие от подхода BoW, при котором каждое слово представляет собой разреженный вектор, *word embedding* - это "плотный" вектор. Классическим алгоритмом, вычисляющим эмбеддинги (то есть "плотные" вектора) слов, является Word2Vec, предложена чешским аспирантом Томашем Миколовым в 2013 году. Эта модель позволяет формировать векторы, которые отражают взаимоотношения между словами: "король" относится к "королеве" так же как "женщина" к "мужчине".

![word_vectors](https://sun9-40.userapi.com/c855436/v855436722/1f3fe6/o5y3vMysSbA.jpg)

Подход Word2Vec основан на интуитивно понятной гипотезе, которая называется гипотезой локальности — "слова, которые встречаются в одинаковых окружениях, имеют близкие значения". Эта гипотеза приводит к двум способам тренировки моделей: *Continious Bag of Words* (когда по контексту предсказываем слово) и *Skip Gram* - когда по слову пытаемся предсказать его контекст. Эмбеддинги, полученные с помощью обоих подходов оказываются идентичными - можно применять любой из них.

Пример контекста: 

*Машинное обучение это* **класс** *методов искусственного интеллекта*

Мы видим, что из текста вырезается окно текста, слово в центре окна мы хотим предсказать, используя слова по краям "окна" (тот самый *контекст*).

На схеме представлены оба подхода:

![word2vec](https://sun9-43.userapi.com/c200816/v200816722/5fb72/o1o8GYCYul0.jpg)

На картинке представлен алгоритм тренировки *W2V*:

![w2v_net](https://sun9-59.userapi.com/c200816/v200816722/5fb7a/mX1xAY2vVgU.jpg)

На схеме слева-направо:

* Входной вектор $(x_1,\ldots,x_v)$ - слово из словаря, закодированное One-Hot
* $W_{V\times N}$ - матрица *word input* -  это эмбеддинги, которые мы обучаем
* Эмбеддинг слова контекста $(h_1,\ldots,h_N)$
* $W`_{N\times V}$ - матрица *word output* -  это тоже эмбеддинги но уже другие (они тоже обучаются в процессе)
* Выходной вектор $(y_1,\ldots,y_V)$ - скор для каждого слова из словаря размерности $V$


Мы видим два матричных перемножения - на самом деле W2V представляет собой очень простую нейронную сеть прямого распространения, *feed forward*. 

На схеме видны две матрицы-скрытые слои. На самом деле это эмбеддинги контента, которые мы обучаем, каждая строка - эмбеддинг размерности N. Матрица эмбеддингов размером (ЧИСЛО СЛОВ В СЛОВАРЕ) X (РАЗМЕРНОСТЬ ЭМБЕДДИНГА) в начале обучения инициализируется рандомными числами, которые “превращаются” в осмысленные эмбеддинги, пока сеть обучается методом обратного распространения ошибки.

На последнем слое мы получаем скоры для каждого слова из словаря. Скор (от англ score) с индексом i - это “уверенность” сети в том, что слово i может быть в контексте слова, которое мы прокидываем через сеть. То есть мы “кормим” сеть контекстом и уменьшаем лосс в случае, когда по контексту правильно удалось распознать слово внутри контекста. Слово с максимальным скором - это предсказание нашей сети. Зная "истинное" слово, которое мы предсказываем и то, что предсказала сеть, мы будем "подкручивать" веса эмбеддингов таким образом, чтобы лосс уменьшался и начинаем все лучше предсказывать слово по контексту.

Ниже показано, как работает, модификация *CBOW* - через нашу "сеть" пропускается каждое слово из контекста, мы пытаемся спрогнозировать слово "внутри" контекста:

![cbow](https://sun9-65.userapi.com/c200816/v200816722/5fb81/oPfs4uHxw6I.jpg)

В питоне существует модуль `gensim` который включает в себя библиотеки для обучения W2V. Этот модуль доступен для установки в [пакете Anaconda](https://anaconda.org/anaconda/gensim)

Давайте применим алгоритм CBOW к нашему тексту:

In [ ]:
from gensim.models import Word2Vec
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

texts = df.tokenized.values

model = Word2Vec(texts, size=10, window=7, min_count=2, workers=4, iter=10, sg=0)

2019-05-05 18:36:26,744 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-05-05 18:36:26,745 : INFO : collecting all words and their counts
2019-05-05 18:36:26,745 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-05-05 18:36:26,759 : INFO : collected 7968 word types from a corpus of 55882 raw words and 4499 sentences
2019-05-05 18:36:26,760 : INFO : Loading a fresh vocabulary
2019-05-05 18:36:26,768 : INFO : effective_min_count=2 retains 3452 unique words (43% of original 7968, drops 4516)
2019-05-05 18:36:26,769 : INFO : effective_min_count=2 leaves 51366 word corpus (91% of original 55882, drops 4516)
2019-05-05 18:36:26,778 : INFO : deleting the raw counts dictionary of 7968 items
2019-05-05 18:36:26,779 : INFO : sample=0.001 downsamples 39 most-common words
2019-05-05 18:36:26,779 : INFO : downsampling leaves estimated 35698 word corpus (69.5% of prior 51366)
2019-05-05 18:36:26,786 : INFO : estimated required mem

Мы обучили эмбеддинги слов. Давайте проверим, какой вектор обучился для слова `android`

In [ ]:
model.wv.get_vector('android')

array([ 0.26981112,  1.2430197 , -3.6010602 ,  0.98848987,  0.6809826 ,
        2.8977017 , -0.6469272 , -0.03298385,  0.0875835 , -0.9362278 ],
      dtype=float32)

Мы видим набор цифр - это вектор длины 10. Давайте найдём, какие слова соответствуют максимально похожим векторам

In [ ]:
model.wv.most_similar('android')

[('blackberry', 0.9917972087860107),
 ('iphone', 0.9915081858634949),
 ('updates', 0.9836416840553284),
 ('shoes', 0.9720171689987183),
 ('condensed', 0.9703693985939026),
 ('christian', 0.968561053276062),
 ('angrybirds', 0.9650596976280212),
 ('whiteboarding', 0.9633722901344299),
 ('yo', 0.9629340767860413),
 ('grill', 0.9623512029647827)]

Мы видим, что модель обучила похожие вектора для слов `blackberry`, `iphone`  - это всё названия телефонов, то есть модель работает!

На основе векторизованных слов можно строить векторное описание целого предложения - такой алгоритм называется `doc2vec`. 